In [1]:
import tensorflow as tf
import numpy as np

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6918000330506361806
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 1453034700
locality {
  bus_id: 1
  links {
  }
}
incarnation: 2352881409656529188
physical_device_desc: "device: 0, name: GeForce GTX 750 Ti, pci bus id: 0000:01:00.0, compute capability: 5.0"
]


In [3]:
mnist = tf.keras.datasets.mnist.load_data()
(x_trainx, y_train), (x_testx, y_test) = mnist

x_train = np.reshape(x_trainx,(60000,784))
x_test = np.reshape(x_testx,(10000,784))

#regulazation
x_train = x_train / 255
x_test = x_test / 255

y_train = np.eye(10)[y_train]
y_test = np.eye(10)[y_test]


#batch
###very important
batch_size = 100
batch_num = 60000 // batch_size
x_train = np.split(np.array(x_train),batch_num)
y_train = np.split(np.array(y_train),batch_num)



In [4]:
#build network
with tf.name_scope('input'):
    x = tf.placeholder(tf.float32,[None,784])
    y = tf.placeholder(tf.float32,[None,10])

#dropout
keep_prob=tf.placeholder(tf.float32)

with tf.name_scope('L1'):
    w1 = tf.Variable(tf.truncated_normal([784,200],stddev=0.1))
    b1 = tf.Variable(tf.zeros([200])+0.1)
    L1 = tf.nn.relu(tf.matmul(x,w1)+b1)
    L1_drop = tf.nn.dropout(L1,keep_prob)

with tf.name_scope('L2'):
    w2 = tf.Variable(tf.truncated_normal([200,200],stddev=0.1))
    b2 = tf.Variable(tf.zeros([200])+0.1)
    L2 = tf.nn.relu(tf.matmul(L1,w2)+b2)
    L2_drop = tf.nn.dropout(L2,keep_prob)

with tf.name_scope('L3'):
    w3 = tf.Variable(tf.truncated_normal([200,10],stddev=0.1))
    b3 = tf.Variable(tf.zeros([10])+0.1)
    L3 = tf.matmul(L2,w3)+b3

with tf.name_scope('softmax'):
    predict = tf.nn.softmax(L3)


#loss
# loss = tf.reduce_mean(tf.square(y - predict))
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=predict))


#gradient
# train_step = tf.train.GradientDescentOptimizer(0.4).minimize(loss)
with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)
# train_step = tf.train.AdamOptimizer(1e-2).minimize(loss)

init = tf.global_variables_initializer()

#accuracy
correct = tf.equal(tf.argmax(y,1),tf.argmax(predict,1))
acc = tf.reduce_mean(tf.cast(correct,tf.float32))


#batch
with tf.Session() as sess:
    sess.run(init)
    writer = tf.summary.FileWriter("logs/",sess.graph)
    for epoch in range(100):
        for batch in range(batch_num):
            sess.run(train_step,feed_dict={x:x_train[batch],y:y_train[batch],keep_prob:0.85})
        with tf.name_scope('acc'):    
            test_accuracy = sess.run(acc, feed_dict={x:x_test,y:y_test})
            train_accuracy = sess.run(acc, feed_dict={x:x_train[0],y:y_train[0],keep_prob:1.0})
            print("test:"+str(test_accuracy)+"  train:" +str(train_accuracy))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
test:0.9416  train:0.95
test:0.9587  train:0.98
test:0.957  train:0.98
test:0.9632  train:0.98
test:0.9656  train:0.98
test:0.9669  train:0.99
test:0.9689  train:0.99
test:0.9746  train:0.99
test:0.9728  train:0.99
test:0.9719  train:0.99
test:0.9762  train:0.99
test:0.9729  train:0.99
test:0.9747  train:0.99
test:0.9738  train:0.99
test:0.9749  train:0.98
test:0.9762  train:0.99
test:0.9736  train:0.99
test:0.9742  train:0.99
test:0.9753  train:0.99
test:0.9783  train:0.99
test:0.9775  train:0.99
test:0.9777  train:0.99
test:0.9758  train:0.99
test:0.978  train:0.99
test:0.9776  train:0.99
test:0.9807  train:0.99
test:0.9756  train:0.99
test:0.9774  train:0.99
test:0.9753  train:0.99
test:0.9797  train:0.99
test:0.9765  train:0.99
test:0.976  train:0.99
test:0.9775  train:0.99
test:0.9783  train:0.99
test:0.9794  train:0.99
test:0.9788  train:0.99
test:0.98  train:0.99
te

KeyboardInterrupt: 